# Вариант 2

### О задании

**В этом домашнем задании** вы разработаете систему машинного перевода без использования параллельных корпусов, выравнивания, механизма внимания, супермощных рекуррентных нейронных сетей глубиной 100500 и других суперпродвинутых технологий.

Однако, даже без параллельных корпусов, эта система может быть достаточно эффективной.

Для нашей системы мы выбираем два родственных славянских языка: украинский и русский.

### Например

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Фрагмент списка Сводеша для некоторых славянских языков

**Список Сводеша** — это лексикостатистический материал. Он назван в честь американского лингвиста Морриса Сводеша и содержит базовую лексику. Этот список используется для определения подгрупп языков, их родства.

Таким образом, мы можем видеть своего рода инвариантность слов для разных славянских языков.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

Но контекстное распределение этих языков демонстрирует еще большую инвариантность. И мы можем использовать этот факт для наших целей.

## Получение данных

In [17]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Скачать вложения можно здесь:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

Загрузите эмбеддинги для русского и украинского.

In [18]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [19]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [ ]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

In [ ]:
uk_emb.most_similar([uk_emb["серпень"]])

In [ ]:
ru_emb.most_similar([uk_emb["серпень"]])

Загрузите небольшие словари для соответствующих пар слов в качестве обучающего и тестового набора.

In [43]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [44]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [91]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Отображение пространства эмбеддингов

Пусть $x_i \in \mathrm{R}^d$ — это распределенное представление слова $i$ на исходном языке, а $y_i \in \mathrm{R}^d$ — векторное представление его перевода. Наша цель — выучить такое линейное преобразование $W$, которое минимизирует евклидово расстояние между $Wx_i$ и $y_i$ для некоторого подмножества эмбеддингов слов. Таким образом, мы можем сформулировать так называемую проблему Прокруста:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
или
$$W^*= \arg\min_W ||WX - Y||_F$$

где $||*||_F$ — норма Фробениуса.

В греческой мифологии Прокруст или "растягиватель" был мошенником-кузнецом и разбойником из Аттики, который атаковал людей, растягивая их или отрезая ноги, чтобы заставить их соответствовать размеру железной кровати. Мы делаем то же самое с исходным пространством встраивания. Наша прокрустева кровать — это целевое пространство встраивания.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

Но подождите... $W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ выглядит как простая множественная линейная регрессия (без intercept fit).

### Линейная регрессия (1 балл)

In [1]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE

Давайте посмотрим на соседей вектора слова _"серпень"_ (_"август"_ на русском) после линейного преобразования.

In [ ]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

Мы видим, что окрестность этого эмбеддинга состоит из разных месяцев, но правильный вариант находится на девятом месте.

В качестве меры качества мы будем использовать percision@1, @5 и @10 (для каждого преобразованного украинского эмбеддинга мы считаем, сколько правильных пар целевого языка найдено в N ближайших соседях в русском пространстве эмбеддингов).

### Оценка качества (1.5 балл)

In [3]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
    precision_val = num_matches / len(pairs)
    return precision_val


In [ ]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [ ]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [ ]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

## Улучшение (ортогональная проблема Прокруста)

Можно показать (см. оригинальную статью), что самосогласованное линейное отображение между семантическими пространствами должно быть ортогональным. 
Мы можем ограничить преобразование $W$ ортогональным. Тогда мы будем решать следующую задачу:

$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

$$I \text{- единичная матрица}$$

Вместо того чтобы создавать еще одну задачу регрессии, мы можем найти оптимальное ортогональное преобразование с использованием сингулярного разложения. Оказывается, что оптимальное преобразование $W^*$ может быть выражено через компоненты SVD:
$$X^TY=U\Sigma V^T\text{, сингулярное разложение}$$
$$W^*=UV^T$$

### Улучшение (2.5 балла)

In [140]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOU CODE HERE

In [141]:
W = learn_transform(X_train, Y_train)

In [ ]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

In [ ]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

## UK-RU Переводчик (5 баллов)

Теперь мы готовы создать простой переводчик на основе слов: для каждого слова на исходном языке в общем пространстве мы находим ближайшее слово на целевом языке.

In [186]:
with open("fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [187]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE HERE

In [ ]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [ ]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

Неплохо, правда? Мы можем легко улучшить перевод, используя языковую модель и не одно, а несколько ближайших соседей в общем пространстве эмбеддингов. Но об этом в следующий раз.